In [9]:
# EJEMPLO TOMADO (PARCIALMENTE) DE https://machinelearningmastery.com/multi-output-regression-models-with-python/

# example of making a prediction with the direct multioutput regression model
from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import LinearSVR
import numpy as np

In [10]:
from sklearn.datasets import make_regression
from sklearn.svm import LinearSVR

# 📌 1. Definimos un dataset sintético de regresión
X, y = make_regression(
    n_samples=1000,      # 1000 muestras (filas de datos)
    n_features=10,       # 10 variables independientes (features por fila)
    n_informative=5,     # solo 5 de esas 10 realmente influyen en la salida (las otras son ruido)
    n_targets=2,         # 2 variables de salida (regresión multi-output)
    random_state=1,      # semilla para reproducibilidad
    noise=0.5            # agregamos ruido gaussiano para que no sea una función perfecta
)

print("Forma de X:", X.shape)  # (1000, 10)
print("Forma de y:", y.shape)  # (1000, 2)

# 📌 2. Definimos el modelo base
# LinearSVR = Support Vector Regression con un kernel lineal
# - Similar a LinearRegression, pero con la lógica de SVM
# - Tolera un margen de error (epsilon) y busca una función que esté lo más plana posible,
#   ignorando errores pequeños pero penalizando los grandes.
model = LinearSVR()

print(model)


Forma de X: (1000, 10)
Forma de y: (1000, 2)
LinearSVR()


In [11]:
from sklearn.multioutput import MultiOutputRegressor

# 📌 1. Creamos el wrapper
# MultiOutputRegressor toma un modelo que normalmente solo predice una salida
# y entrena una copia de ese modelo para cada columna (target) de y.
# En tu caso: y tiene 2 columnas → se entrenarán 2 modelos LinearSVR independientes.
wrapper = MultiOutputRegressor(model)

# 📌 2. Entrenamos el modelo sobre TODO el dataset
# X → features de entrada (1000 filas × 10 columnas)
# y → targets (1000 filas × 2 columnas)
wrapper.fit(X, y)

print("Wrapper entrenado con 2 modelos SVR internos (uno por cada salida).")


Wrapper entrenado con 2 modelos SVR internos (uno por cada salida).


In [12]:
# make a single prediction
row = [0.21947749, 0.32948997, 0.81560036, 0.440956, -0.0606303, -0.29257894, -0.2820059, -0.00290545, 0.96402263, 0.04992249]
yhat = wrapper.predict([row])
# summarize the prediction
print('Predicted: %s' % yhat[0])
print(yhat)

Predicted: [50.01498545 64.50948203]
[[50.01498545 64.50948203]]


In [ ]:
# 📌 Diferencia clave

# wrapper.get_params() → parámetros del envoltorio multi-output + los del modelo base (con prefijo).

# model.get_params() → parámetros solo del modelo base (LinearSVR).

# 📌 1. Obtener parámetros del wrapper (MultiOutputRegressor)
z = wrapper.get_params()
print("Parámetros del wrapper (MultiOutputRegressor):\n", z)

# 📌 2. Obtener parámetros del modelo base (LinearSVR)
v = model.get_params()
print("\nParámetros del modelo base (LinearSVR):\n", v)


{'estimator__C': 1.0, 'estimator__dual': 'auto', 'estimator__epsilon': 0.0, 'estimator__fit_intercept': True, 'estimator__intercept_scaling': 1.0, 'estimator__loss': 'epsilon_insensitive', 'estimator__max_iter': 1000, 'estimator__random_state': None, 'estimator__tol': 0.0001, 'estimator__verbose': 0, 'estimator': LinearSVR(), 'n_jobs': None}
{'C': 1.0, 'dual': 'auto', 'epsilon': 0.0, 'fit_intercept': True, 'intercept_scaling': 1.0, 'loss': 'epsilon_insensitive', 'max_iter': 1000, 'random_state': None, 'tol': 0.0001, 'verbose': 0}


In [ ]:
# 📌 Guardar los diccionarios como archivos .npy
# np.save guarda objetos en formato binario de NumPy
np.save('file1.npy', z)   # guarda el dict z (parámetros del wrapper)
np.save('file2.npy', v)   # guarda el dict v (parámetros del modelo base)


In [17]:
# 📌 1. Cargar el archivo .npy
new_dict = np.load('file1.npy', allow_pickle=True)
print(new_dict)
print(type(new_dict))   # → verás que es un numpy.ndarray con un solo elemento (el dict)

# 📌 2. Convertir el array en diccionario
# Tu código hace dict(enumerate(...)) → genera un diccionario de índice:valor
# Pero en realidad lo que queremos es extraer el objeto que está dentro del array
d = dict(enumerate(new_dict.flatten(), 1))
print(d)
print(type(d))          # → es un dict con 1 entrada: {1: {...}}

# 📌 3. Acceder al diccionario real
par = d[1]              # obtenés el verdadero diccionario de parámetros
print(par)


{'estimator__C': 1.0, 'estimator__dual': 'auto', 'estimator__epsilon': 0.0, 'estimator__fit_intercept': True, 'estimator__intercept_scaling': 1.0, 'estimator__loss': 'epsilon_insensitive', 'estimator__max_iter': 1000, 'estimator__random_state': None, 'estimator__tol': 0.0001, 'estimator__verbose': 0, 'estimator': LinearSVR(), 'n_jobs': None}
<class 'numpy.ndarray'>
{1: {'estimator__C': 1.0, 'estimator__dual': 'auto', 'estimator__epsilon': 0.0, 'estimator__fit_intercept': True, 'estimator__intercept_scaling': 1.0, 'estimator__loss': 'epsilon_insensitive', 'estimator__max_iter': 1000, 'estimator__random_state': None, 'estimator__tol': 0.0001, 'estimator__verbose': 0, 'estimator': LinearSVR(), 'n_jobs': None}}
<class 'dict'>
{'estimator__C': 1.0, 'estimator__dual': 'auto', 'estimator__epsilon': 0.0, 'estimator__fit_intercept': True, 'estimator__intercept_scaling': 1.0, 'estimator__loss': 'epsilon_insensitive', 'estimator__max_iter': 1000, 'estimator__random_state': None, 'estimator__tol':

In [ ]:
from sklearn.svm import LinearSVR
from sklearn.multioutput import MultiOutputRegressor

# 📌 1. Creamos un nuevo modelo igual que antes
model1 = MultiOutputRegressor(LinearSVR())

# 📌 2. Le cargamos los parámetros guardados
# par es un diccionario con hiperparámetros (ej: C, epsilon, max_iter, etc.)
# set_params(**par) actualiza la configuración del modelo con esos valores
model1.set_params(**par)

# 📌 3. Entrenamos el modelo desde cero
# Ojo: aunque tenga los mismos hiperparámetros, el entrenamiento vuelve a hacerse
# sobre los datos originales (X, y).
model1.fit(X, y)

# 📌 4. Hacemos una predicción sobre la fila 'row'
print("Predicción con el nuevo modelo entrenado:", model1.predict([row]))


[[50.0151902  64.50285243]]


# Luego de esta prueba, la conclusión es la sgte: los métodos set_params y get_params NO GUARDAN los modelos entrenados. Solamente guardan en un diccionario los parámetros del modelo, para que luego se puedan resguardar en un archivo y cargar posteriormente. Para que el modelo sea funcional, hay que entrenarlo nuevamente.